# Getting MAL anime-lists
* We collect the anime-list for each username in `data/mal/user_facts/usernames.txt` and `data/mal/user_facts/recent_usernames.txt` and `data/mal/user_facts/usernames_from_id.txt`
* You can terminate or restart the notebook at any point without losing progress. All anime-lists found so far will be stored at `data/mal/user_anime_facts/user_anime_list.csv` 
* This notebook will run indefinitely. You must manually terminate once an acceptable number of anime-lists have been found

In [ ]:
import datetime
import json
import logging
import os
import random
import time

import numpy as np
import pandas as pd
import requests
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm

## Basic setup

In [ ]:
# outdir
data_path = "../../data/mal/user_anime_facts"
if not os.path.exists(data_path):
    os.mkdir(data_path)
os.chdir(data_path)

In [ ]:
# logging
logger = logging.getLogger("GetUserAnimeLists")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(name)s:%(levelname)s:%(asctime)s: %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
)
for stream in [
    logging.FileHandler("get_user_anime_lists.log"),
    logging.StreamHandler(),
]:
    stream.setFormatter(formatter)
    logger.addHandler(stream)

## Parse MAL API

In [ ]:
token = json.load(open("../mal_authentication/token.json", "r"))

In [ ]:
# apply rate limiting with exponential backoff for unexpected errors
@sleep_and_retry
@limits(calls=1, period=0.75)
def call_api(url, retry_timeout=1):
    try:
        response = requests.get(
            url, headers={"Authorization": f'Bearer {token["access_token"]}'}
        )
        if response.status_code in [500, 504] and retry_timeout < 3600:
            # This can occur if MAL servers go down or if the page doesnt exist
            raise Exception(f"{response.status_code}")
    except Exception as e:
        logger.warning(
            f"Received error {str(e)} while accessing {url}. Retrying in {retry_timeout} seconds"
        )
        time.sleep(retry_timeout)
        retry_timeout = min(retry_timeout * 2, 3600)
        return call_api(url, retry_timeout)
    return response

In [ ]:
# utilities for extracting an anime list from the MAL API


def parse_json_node(x):
    ls = x["list_status"]
    entry = pd.DataFrame.from_dict(
        {
            "uid": [x["node"]["id"]],
            "status": [ls.get("status", "")],
            "score": [ls.get("score", -1)],
            "num_episodes_watched": [ls.get("num_episodes_watched", -1)],
            "is_rewatching": [ls.get("is_rewatching", False)],
            "start_date": [ls.get("start_date", "")],
            "finish_date": [ls.get("finish_date", "")],
            "priority": [ls.get("priority", -1)],
            "num_times_rewatched": [ls.get("num_times_rewatched", -1)],
            "rewatch_value": [ls.get("rewatch_value", -1)],
            "updated_at": [ls.get("updated_at", "")],
        }
    )
    return entry


def process_json(json):
    entries = [parse_json_node(x) for x in json["data"]]
    if entries:
        return pd.concat(entries, ignore_index=True)
    else:
        return pd.DataFrame.from_dict(
            {
                "uid": [],
                "status": [],
                "score": [],
                "num_episodes_watched": [],
                "is_rewatching": [],
                "start_date": [],
                "finish_date": [],
                "priority": [],
                "num_times_rewatched": [],
                "rewatch_value": [],
                "updated_at": [],
            }
        )


def get_user_anime_list(username):
    anime_lists = []
    more_pages = True
    url = f"https://api.myanimelist.net/v2/users/{username}/animelist?limit=1000&fields=list_status&nsfw=true"
    while more_pages:
        response = call_api(url)
        if response.status_code in [403, 404]:
            # 403: This can occur if the user privated their list
            # 404: This can occur if the user deleted their account
            return pd.DataFrame(), False
        if not response.ok:
            logger.warning(f"Error {response} received when handling {url}")
            return pd.DataFrame(), False

        json = response.json()
        anime_lists.append(process_json(json))
        more_pages = "next" in json["paging"]
        if more_pages:
            url = json["paging"]["next"]
    user_anime_list = pd.concat(anime_lists, ignore_index=True)
    user_anime_list["username"] = username
    return user_anime_list, True

## Sort users by recency

In [ ]:
def read_usernames_from_path(base):
    path = f"../user_facts/{base}.txt"
    if not os.path.exists(path):
        return set()
    with open(path, "r") as f:
        return {x.strip() for x in f.readlines()}


def read_usernames():
    usernames = set()
    usernames |= read_usernames_from_path("usernames")
    usernames |= read_usernames_from_path("recent_usernames")
    usernames |= read_usernames_from_path("usernames_from_id")
    return list(usernames)

In [ ]:
def read_user_status():
    user_status_file = "user_status.csv"
    if os.path.exists(user_status_file):
        return pd.read_csv(user_status_file, keep_default_na=False)
    else:
        return pd.DataFrame.from_dict(
            {
                "username": [],
                "access_timestamp": [],
                "success": [],
            }
        )

In [ ]:
def prioritize_users():
    usernames = read_usernames()
    user_status = read_user_status()
    new_users = list(set(usernames) - set(user_status["username"]))
    random.shuffle(new_users)
    existing_users = list(user_status.sort_values(by="access_timestamp")["username"])
    logger.info(
        f"Getting the anime lists of {len(new_users)} new users and refreshing "
        f"the anime lists of {len(existing_users)} existing users!"
    )
    return new_users + existing_users

## Continuously refresh anime lists
* We take the least recently refreshed users and refresh their anime lists
* These anime lists are stored in a temporary block
* Once the block is big enough, we atomically merge it with the existing anime lists

In [ ]:
def merge_block(file, user_field, users):
    outfile = file + "~"
    blockfile = file + ".block"
    first_run = not os.path.exists(file)
    with open(outfile, "w") as out_file:
        # copy over all the unchaged users
        if not first_run:
            with open(file, "r") as in_file:
                header = False
                for line in tqdm(in_file):
                    if not header:
                        header = True
                        out_file.write(line)
                        continue
                    fields = line.strip().split(",")
                    if fields[user_field] not in users:
                        out_file.write(line)

        # copy over the new block
        with open(blockfile, "r") as in_file:
            header = False
            for line in tqdm(in_file):
                if not header:
                    if first_run:
                        out_file.write(line)
                    header = True
                    continue
                out_file.write(line)
    os.replace(outfile, file)

In [ ]:
def merge_blocks():
    users = set(pd.read_csv("user_status.csv.block", keep_default_na=False)["username"])
    merge_block("user_anime_list.csv", -1, users)
    merge_block("user_status.csv", 0, users)
    logger.info(f"Merging block of {len(users)} users into the main database")

In [ ]:
# get the anime list for each new user and write to disk
while True:
    usernames = prioritize_users()[:50000]
    block = set()
    for username in tqdm(usernames):
        user_anime_list, ok = get_user_anime_list(username)
        user_anime_list.to_csv(
            "user_anime_list.csv.block",
            index=False,
            mode="w" if not block else "a+",
            header=not block,
        )
        
        user_status_entry = pd.DataFrame.from_dict(
            {
                "username": [username],
                "access_timestamp": [int(datetime.datetime.now().timestamp())],
                "success": [ok],
            }
        )
        user_status_entry.to_csv(
            "user_status.csv.block",
            index=False,
            mode="w" if not block else "a+",
            header=not block,
        )
        block.add(username)
    merge_blocks()